In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import re


model_name = "eryk-mazus/polka-1.1b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": "cpu"}  )

streamer = TextStreamer(tokenizer, skip_prompt=True)

system_prompt = "Działasz jak kalkulator. Obliczasz poprawne wyniki działań podanych na wejście."
chat = [{"role": "system", "content": system_prompt}]

# Compose a short song on programming.
user_input = "2+3="
chat.append({"role": "user", "content": user_input})

# Generate - add_generation_prompt to make sure it continues as assistant
inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt")
# For multi-GPU, find the device of the first parameter of the model
first_param_device = next(model.parameters()).device
inputs = inputs.to(first_param_device)

with torch.no_grad():
    outputs = model.generate(
        inputs,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=30,
        temperature=0.2,
        repetition_penalty=1.15,
        top_p=0.95,
        do_sample=True,
        streamer=streamer,
    )

# Add just the new tokens to our chat
new_tokens = outputs[0, inputs.size(1):]
response = tokenizer.decode(new_tokens, skip_special_tokens=True)
chat.append({"role": "assistant", "content": response})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

To jest podstawowa zasada matematyki, która mówi, że suma dwóch liczb nie może być mniejsza od


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import re

model_name = "eryk-mazus/polka-1.1b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": "cpu"}
)


streamer = TextStreamer(tokenizer, skip_prompt=True)


system_prompt = "Działasz jak kalkulator. Obliczasz poprawne wyniki działań podanych na wejście."
chat = [{"role": "system", "content": system_prompt}]


def detect_category(question):
    if re.search(r'\d+\s*\+\s*\d+', question):
        return "dodawanie"
    elif re.search(r'\d+\s*-\s*\d+', question):
        return "odejmowanie"
    elif re.search(r'\d+\s*\*\s*\d+', question):
        return "mnożenie"
    elif re.search(r'\d+\s*/\s*\d+', question):
        return "dzielenie"
    else:
        return "inne"


def get_few_shot_examples(category):
    if category == "dodawanie":
        return [
            {"role": "user", "content": "2 + 3 ="},
            {"role": "assistant", "content": "5"},
            {"role": "user", "content": "10 + 15 ="},
            {"role": "assistant", "content": "25"},
            {"role": "user", "content": "7 + 8 ="},
            {"role": "assistant", "content": "15"},
            {"role": "user", "content": "100 + 200 ="},
            {"role": "assistant", "content": "300"}
        ]
    elif category == "odejmowanie":
        return [
            {"role": "user", "content": "5 - 2 ="},
            {"role": "assistant", "content": "3"},
            {"role": "user", "content": "20 - 4 ="},
            {"role": "assistant", "content": "16"},
            {"role": "user", "content": "100 - 50 ="},
            {"role": "assistant", "content": "50"},
            {"role": "user", "content": "80 - 30 ="},
            {"role": "assistant", "content": "50"}
        ]
    elif category == "mnożenie":
        return [
            {"role": "user", "content": "3 * 3 ="},
            {"role": "assistant", "content": "9"},
            {"role": "user", "content": "5 * 6 ="},
            {"role": "assistant", "content": "30"},
            {"role": "user", "content": "7 * 8 ="},
            {"role": "assistant", "content": "56"},
            {"role": "user", "content": "10 * 10 ="},
            {"role": "assistant", "content": "100"}
        ]
    elif category == "dzielenie":
        return [
            {"role": "user", "content": "6 / 2 ="},
            {"role": "assistant", "content": "3"},
            {"role": "user", "content": "15 / 3 ="},
            {"role": "assistant", "content": "5"},
            {"role": "user", "content": "100 / 4 ="},
            {"role": "assistant", "content": "25"},
            {"role": "user", "content": "9 / 3 ="},
            {"role": "assistant", "content": "3"}
        ]
    else:
        return []


def test_model_on_questions(kategorie_pytania, kategorie_odpowiedzi):
    poprawne_odpowiedzi = 0
    total_questions = 0

    for category_pytania, category_odpowiedzi in zip(kategorie_pytania, kategorie_odpowiedzi):
        for pytanie, poprawna_odpowiedz in zip(category_pytania, category_odpowiedzi):
            chat = [{"role": "system", "content": system_prompt}]
            category = detect_category(pytanie)
            few_shot_examples = get_few_shot_examples(category)
            chat.extend(few_shot_examples)

            if category in ["dodawanie", "odejmowanie", "mnożenie", "dzielenie"]:
                chat.append({"role": "user", "content": pytanie})


                inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt")
                inputs = inputs.to("cpu")

                with torch.no_grad():
                    outputs = model.generate(
                        inputs,
                        pad_token_id=tokenizer.eos_token_id,
                        max_new_tokens=10,
                        temperature=0.1,
                        repetition_penalty=1.15,
                        top_p=0.95,
                        do_sample=False,
                        streamer=streamer,
                    )

                new_tokens = outputs[0, inputs.size(1):]
                response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

            else:
                response = "Nie obsługuję tego typu pytań."


            print(f"Pytanie: {pytanie}")
            print(f"Odpowiedź modelu: {response}")
            print(f"Poprawna odpowiedź: {poprawna_odpowiedz}")
            print("-------------------------------")

            if poprawna_odpowiedz.lower() in response.lower():
                poprawne_odpowiedzi += 1

            total_questions += 1

    procent_poprawnych = (poprawne_odpowiedzi / total_questions) * 100
    print(f"Procent poprawnych odpowiedzi: {procent_poprawnych:.2f}%")

kategorie_pytania_test = [

    [
        "2 + 3 =", "10 + 15 =", "100 + 200 =", "7 + 8 =", "50 + 25 =",
        "9 + 6 =", "13 + 12 =", "1000 + 500 =", "20 + 30 =", "500 + 250 =",
        "6 + 14 =", "33 + 77 =", "99 + 101 =", "23 + 45 =", "70 + 25 =",
        "15 + 35 =", "60 + 90 =", "1 + 999 =", "250 + 350 =", "88 + 22 ="
    ],

    [
        "5 - 2 =", "20 - 4 =", "80 - 30 =", "50 - 25 =", "100 - 55 =",
        "22 - 11 =", "70 - 20 =", "90 - 45 =", "300 - 100 =", "15 - 7 =",
        "40 - 18 =", "60 - 30 =", "120 - 90 =", "100 - 75 =", "18 - 9 =",
        "55 - 25 =", "35 - 15 =", "500 - 250 =", "77 - 33 =", "99 - 22 ="
    ],

    [
        "3 * 3 =", "5 * 6 =", "7 * 8 =", "10 * 10 =", "4 * 4 =",
        "6 * 7 =", "9 * 5 =", "12 * 12 =", "8 * 9 =", "11 * 11 =",
        "15 * 2 =", "13 * 4 =", "16 * 3 =", "14 * 5 =", "18 * 6 =",
        "20 * 20 =", "25 * 4 =", "9 * 9 =", "19 * 2 =", "21 * 3 ="
    ],

    [
        "6 / 2 =", "15 / 3 =", "100 / 4 =", "30 / 5 =", "20 / 4 =",
        "25 / 5 =", "60 / 6 =", "80 / 8 =", "99 / 3 =", "50 / 2 =",
        "18 / 6 =", "200 / 10 =", "120 / 4 =", "45 / 9 =", "72 / 8 =",
        "90 / 3 =", "1000 / 10 =", "500 / 25 =", "64 / 16 =", "144 / 12 ="
    ]
]

kategorie_odpowiedzi_test = [

    [
        "5", "25", "300", "15", "75",
        "15", "25", "1500", "50", "750",
        "20", "110", "200", "68", "95",
        "50", "150", "1000", "600", "110"
    ],

    [
        "3", "16", "50", "25", "45",
        "11", "50", "45", "200", "8",
        "22", "30", "30", "25", "9",
        "30", "20", "250", "44", "77"
    ],

    [
        "9", "30", "56", "100", "16",
        "42", "45", "144", "72", "121",
        "30", "52", "48", "70", "108",
        "400", "100", "81", "38", "63"
    ],

    [
        "3", "5", "25", "6", "5",
        "5", "10", "10", "33", "25",
        "3", "20", "30", "5", "9",
        "30", "100", "20", "4", "12"
    ]
]



test_model_on_questions(kategorie_pytania_test, kategorie_odpowiedzi_test)


4 
6 
8 
1
Pytanie: 2 + 3 =
Odpowiedź modelu: 4 
6 
8 
1
Poprawna odpowiedź: 5
-------------------------------
25 

Czyli, jeśli
Pytanie: 10 + 15 =
Odpowiedź modelu: 25 

Czyli, jeśli
Poprawna odpowiedź: 25
-------------------------------
400 

Czy mogę prosi
Pytanie: 100 + 200 =
Odpowiedź modelu: 400 

Czy mogę prosi
Poprawna odpowiedź: 300
-------------------------------
9 
64 

Wygląda
Pytanie: 7 + 8 =
Odpowiedź modelu: 9 
64 

Wygląda
Poprawna odpowiedź: 15
-------------------------------
105 

Czy mogę prosi
Pytanie: 50 + 25 =
Odpowiedź modelu: 105 

Czy mogę prosi
Poprawna odpowiedź: 75
-------------------------------
15 

Czy mogę prosić
Pytanie: 9 + 6 =
Odpowiedź modelu: 15 

Czy mogę prosić
Poprawna odpowiedź: 15
-------------------------------
25 

Czyli, jeśli
Pytanie: 13 + 12 =
Odpowiedź modelu: 25 

Czyli, jeśli
Poprawna odpowiedź: 25
-------------------------------
2000 

Czy mogę
Pytanie: 1000 + 500 =
Odpowiedź modelu: 2000 

Czy mogę
Poprawna odpowiedź: 1500
-----------

In [ ]:
kategorie_pytania_stale = [

    [
        "2 + 1 =", "2 + 3 =", "2 + 5 =", "2 + 10 =", "2 + 15 =",
        "2 + 20 =", "2 + 25 =", "2 + 30 =", "2 + 50 =", "2 + 100 =",
        "2 + 200 =", "2 + 7 =", "2 + 8 =", "2 + 4 =", "2 + 6 =",
        "2 + 12 =", "2 + 9 =", "2 + 11 =", "2 + 13 =", "2 + 14 ="
    ],

    [
        "20 - 1 =", "20 - 3 =", "20 - 5 =", "20 - 10 =", "20 - 15 =",
        "20 - 20 =", "20 - 2 =", "20 - 7 =", "20 - 8 =", "20 - 4 =",
        "20 - 6 =", "20 - 12 =", "20 - 9 =", "20 - 11 =", "20 - 14 =",
        "20 - 13 =", "20 - 16 =", "20 - 18 =", "20 - 19 =", "20 - 17 ="
    ],

    [
        "1 * 2 =", "3 * 2 =", "5 * 2 =", "10 * 2 =", "15 * 2 =",
        "20 * 2 =", "25 * 2 =", "30 * 2 =", "50 * 2 =", "100 * 2 =",
        "200 * 2 =", "7 * 2 =", "8 * 2 =", "4 * 2 =", "6 * 2 =",
        "12 * 2 =", "9 * 2 =", "11 * 2 =", "13 * 2 =", "14 * 2 ="
    ],

    [
        "2 / 2 =", "4 / 2 =", "6 / 2 =", "8 / 2 =", "10 / 2 =",
        "12 / 2 =", "14 / 2 =", "16 / 2 =", "20 / 2 =", "40 / 2 =",
        "80 / 2 =", "100 / 2 =", "60 / 2 =", "24 / 2 =", "18 / 2 =",
        "30 / 2 =", "50 / 2 =", "70 / 2 =", "90 / 2 =", "200 / 2 ="
    ]
]

kategorie_odpowiedzi_stale = [

    [
        "3", "5", "7", "12", "17",
        "22", "27", "32", "52", "102",
        "202", "9", "10", "6", "8",
        "14", "11", "13", "15", "16"
    ],

    [
        "19", "17", "15", "10", "5",
        "0", "18", "13", "12", "16",
        "14", "8", "11", "9", "6",
        "7", "4", "2", "1", "3"
    ],

    [
        "2", "6", "10", "20", "30",
        "40", "50", "60", "100", "200",
        "400", "14", "16", "8", "12",
        "24", "18", "22", "26", "28"
    ],

    [
        "1", "2", "3", "4", "5",
        "6", "7", "8", "10", "20",
        "40", "50", "30", "12", "9",
        "15", "25", "35", "45", "100"
    ]
]

test_model_on_questions(kategorie_pytania_stale, kategorie_odpowiedzi_stale)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


4 

Czyli, jeśli ch
Pytanie: 2 + 1 =
Odpowiedź modelu: 4 

Czyli, jeśli ch
Poprawna odpowiedź: 3
-------------------------------
4 
6 
8 
1
Pytanie: 2 + 3 =
Odpowiedź modelu: 4 
6 
8 
1
Poprawna odpowiedź: 5
-------------------------------
4 
6 
8 
1
Pytanie: 2 + 5 =
Odpowiedź modelu: 4 
6 
8 
1
Poprawna odpowiedź: 7
-------------------------------
4 

Mam nadzieję, że
Pytanie: 2 + 10 =
Odpowiedź modelu: 4 

Mam nadzieję, że
Poprawna odpowiedź: 12
-------------------------------
4 

Mam nadzieję, że
Pytanie: 2 + 15 =
Odpowiedź modelu: 4 

Mam nadzieję, że
Poprawna odpowiedź: 17
-------------------------------
4 

Mam nadzieję, że
Pytanie: 2 + 20 =
Odpowiedź modelu: 4 

Mam nadzieję, że
Poprawna odpowiedź: 22
-------------------------------
4 

Mam nadzieję, że
Pytanie: 2 + 25 =
Odpowiedź modelu: 4 

Mam nadzieję, że
Poprawna odpowiedź: 27
-------------------------------
4 

Mam nadzieję, że
Pytanie: 2 + 30 =
Odpowiedź modelu: 4 

Mam nadzieję, że
Poprawna odpowiedź: 32
----------------

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import re

model_name = "eryk-mazus/polka-1.1b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": "cpu"}
)

streamer = TextStreamer(tokenizer, skip_prompt=True)

system_prompt = "Działasz jak kalkulator. Obliczasz poprawne wyniki działań podanych na wejście."
chat = [{"role": "system", "content": system_prompt}]

def detect_category(question):
    if re.search(r'\d+\s*\+\s*\d+', question):
        return "dodawanie"
    elif re.search(r'\d+\s*-\s*\d+', question):
        return "odejmowanie"
    elif re.search(r'\d+\s*\*\s*\d+', question):
        return "mnożenie"
    elif re.search(r'\d+\s*/\s*\d+', question):
        return "dzielenie"
    else:
        return "inne"

def get_few_shot_examples(category):
    if category == "dodawanie":
        return [
            {"role": "user", "content": "Działanie: 2 + 3 = Wynik: "},
            {"role": "assistant", "content": "5"},
            {"role": "user", "content": "Działanie: 10 + 15 = Wynik: "},
            {"role": "assistant", "content": "25"},
            {"role": "user", "content": "Działanie: 7 + 8 = Wynik: "},
            {"role": "assistant", "content": "15"},
            {"role": "user", "content": "Działanie: 100 + 200 = Wynik: "},
            {"role": "assistant", "content": "300"}
        ]
    elif category == "odejmowanie":
        return [
            {"role": "user", "content": "Działanie: 5 - 2 = Wynik: "},
            {"role": "assistant", "content": "3"},
            {"role": "user", "content": "Działanie: 20 - 4 = Wynik: "},
            {"role": "assistant", "content": "16"},
            {"role": "user", "content": "Działanie: 100 - 50 = Wynik: "},
            {"role": "assistant", "content": "50"},
            {"role": "user", "content": "Działanie: 80 - 30 = Wynik: "},
            {"role": "assistant", "content": "50"}
        ]
    elif category == "mnożenie":
        return [
            {"role": "user", "content": "Działanie: 3 * 3 = Wynik: "},
            {"role": "assistant", "content": "9"},
            {"role": "user", "content": "Działanie: 5 * 6 = Wynik: "},
            {"role": "assistant", "content": "30"},
            {"role": "user", "content": "Działanie: 7 * 8 = Wynik: "},
            {"role": "assistant", "content": "56"},
            {"role": "user", "content": "Działanie: 10 * 10 = Wynik: "},
            {"role": "assistant", "content": "100"}
        ]
    elif category == "dzielenie":
        return [
            {"role": "user", "content": "Działanie: 6 / 2 = Wynik: "},
            {"role": "assistant", "content": "3"},
            {"role": "user", "content": "Działanie: 15 / 3 = Wynik: "},
            {"role": "assistant", "content": "5"},
            {"role": "user", "content": "Działanie: 100 / 4 = Wynik: "},
            {"role": "assistant", "content": "25"},
            {"role": "user", "content": "Działanie: 9 / 3 = Wynik: "},
            {"role": "assistant", "content": "3"}
        ]
    else:
        return []

def test_model_on_questions(kategorie_pytania, kategorie_odpowiedzi):
    poprawne_odpowiedzi = 0
    total_questions = 0

    for category_pytania, category_odpowiedzi in zip(kategorie_pytania, kategorie_odpowiedzi):
        for pytanie, poprawna_odpowiedz in zip(category_pytania, category_odpowiedzi):
            chat = [{"role": "system", "content": system_prompt}]
            category = detect_category(pytanie)
            few_shot_examples = get_few_shot_examples(category)
            chat.extend(few_shot_examples)

            if category in ["dodawanie", "odejmowanie", "mnożenie", "dzielenie"]:
                # Formatowanie pytania z wymaganym tekstem
                formatted_question = f"Działanie: {pytanie.strip()} Wynik: "
                chat.append({"role": "user", "content": formatted_question})

                inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt")
                inputs = inputs.to("cpu")

                with torch.no_grad():
                    outputs = model.generate(
                        inputs,
                        pad_token_id=tokenizer.eos_token_id,
                        max_new_tokens=10,
                        temperature=0.1,
                        repetition_penalty=1.15,
                        top_p=0.95,
                        do_sample=False,
                        streamer=streamer,
                    )

                new_tokens = outputs[0, inputs.size(1):]
                response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

            else:
                response = "Nie obsługuję tego typu pytań."

            print(f"Pytanie: {formatted_question}")
            print(f"Odpowiedź modelu: {response}")
            print(f"Poprawna odpowiedź: {poprawna_odpowiedz}")
            print("-------------------------------")

            if poprawna_odpowiedz.lower() in response.lower():
                poprawne_odpowiedzi += 1

            total_questions += 1

    procent_poprawnych = (poprawne_odpowiedzi / total_questions) * 100
    print(f"Procent poprawnych odpowiedzi: {procent_poprawnych:.2f}%")

kategorie_pytania_test = [

    [
        "2 + 3 =", "10 + 15 =", "100 + 200 =", "7 + 8 =", "50 + 25 =",
        "9 + 6 =", "13 + 12 =", "1000 + 500 =", "20 + 30 =", "500 + 250 =",
        "6 + 14 =", "33 + 77 =", "99 + 101 =", "23 + 45 =", "70 + 25 =",
        "15 + 35 =", "60 + 90 =", "1 + 999 =", "250 + 350 =", "88 + 22 ="
    ],

    [
        "5 - 2 =", "20 - 4 =", "80 - 30 =", "50 - 25 =", "100 - 55 =",
        "22 - 11 =", "70 - 20 =", "90 - 45 =", "300 - 100 =", "15 - 7 =",
        "40 - 18 =", "60 - 30 =", "120 - 90 =", "100 - 75 =", "18 - 9 =",
        "55 - 25 =", "35 - 15 =", "500 - 250 =", "77 - 33 =", "99 - 22 ="
    ],

    [
        "3 * 3 =", "5 * 6 =", "7 * 8 =", "10 * 10 =", "4 * 4 =",
        "6 * 7 =", "9 * 5 =", "12 * 12 =", "8 * 9 =", "11 * 11 =",
        "15 * 2 =", "13 * 4 =", "16 * 3 =", "14 * 5 =", "18 * 6 =",
        "20 * 20 =", "25 * 4 =", "9 * 9 =", "19 * 2 =", "21 * 3 ="
    ],

    [
        "6 / 2 =", "15 / 3 =", "100 / 4 =", "30 / 5 =", "20 / 4 =",
        "25 / 5 =", "60 / 6 =", "80 / 8 =", "99 / 3 =", "50 / 2 =",
        "18 / 6 =", "200 / 10 =", "120 / 4 =", "45 / 9 =", "72 / 8 =",
        "90 / 3 =", "1000 / 10 =", "500 / 25 =", "64 / 16 =", "144 / 12 ="
    ]
]

kategorie_odpowiedzi_test = [

    [
        "5", "25", "300", "15", "75",
        "15", "25", "1500", "50", "750",
        "20", "110", "200", "68", "95",
        "50", "150", "1000", "600", "110"
    ],

    [
        "3", "16", "50", "25", "45",
        "11", "50", "45", "200", "8",
        "22", "30", "30", "25", "9",
        "30", "20", "250", "44", "77"
    ],

    [
        "9", "30", "56", "100", "16",
        "42", "45", "144", "72", "121",
        "30", "52", "48", "70", "108",
        "400", "100", "81", "38", "63"
    ],

    [
        "3", "5", "25", "6", "5",
        "5", "10", "10", "33", "25",
        "3", "20", "30", "5", "9",
        "30", "100", "20", "4", "12"
    ]
]

test_model_on_questions(kategorie_pytania_test, kategorie_odpowiedzi_test)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


6 
Całkowity wynik dział
Pytanie: Działanie: 2 + 3 = Wynik: 
Odpowiedź modelu: 6 
Całkowity wynik dział
Poprawna odpowiedź: 5
-------------------------------
45 

Zatem, wynik
Pytanie: Działanie: 10 + 15 = Wynik: 
Odpowiedź modelu: 45 

Zatem, wynik
Poprawna odpowiedź: 25
-------------------------------
400 

Zatem, wyn
Pytanie: Działanie: 100 + 200 = Wynik: 
Odpowiedź modelu: 400 

Zatem, wyn
Poprawna odpowiedź: 300
-------------------------------
45 

Zatem, wynik
Pytanie: Działanie: 7 + 8 = Wynik: 
Odpowiedź modelu: 45 

Zatem, wynik
Poprawna odpowiedź: 15
-------------------------------
105 

Mam nadzie
Pytanie: Działanie: 50 + 25 = Wynik: 
Odpowiedź modelu: 105 

Mam nadzie
Poprawna odpowiedź: 75
-------------------------------
45 

Zatem, wynik
Pytanie: Działanie: 9 + 6 = Wynik: 
Odpowiedź modelu: 45 

Zatem, wynik
Poprawna odpowiedź: 15
-------------------------------
6 

Zatem, wynik dział
Pytanie: Działanie: 13 + 12 = Wynik: 
Odpowiedź modelu: 6 

Zatem, wynik dział
Poprawna o

In [ ]:

test_model_on_questions(kategorie_pytania_stale, kategorie_odpowiedzi_stale)

<|im_start|> system
Działasz jak kalkulator. Obliczasz poprawne wyniki działań podanych na wejście.<|im_end|> 
<|im_start|> user
2 + 3 =<|im_end|> 
<|im_start|> assistant
5<|im_end|> 
<|im_start|> user
10 + 15 =<|im_end|> 
<|im_start|> assistant
25<|im_end|> 
<|im_start|> user
7 + 8 =<|im_end|> 
<|im_start|> assistant
15<|im_end|> 
<|im_start|> user
100 + 200 =<|im_end|> 
<|im_start|> assistant
300<|im_end|> 
<|im_start|> user
2 + 1 =<|im_end|> 
<|im_start|> assistant
4 

Czyli, jeśli ch
Pytanie: 2 + 1 =
Odpowiedź modelu: 4 

Czyli, jeśli ch
Poprawna odpowiedź: 3
-------------------------------
4 
6 
8 
1
Pytanie: 2 + 3 =
Odpowiedź modelu: 4 
6 
8 
1
Poprawna odpowiedź: 5
-------------------------------
4 
6 
8 
1
Pytanie: 2 + 5 =
Odpowiedź modelu: 4 
6 
8 
1
Poprawna odpowiedź: 7
-------------------------------
4 

Mam nadzieję, że
Pytanie: 2 + 10 =
Odpowiedź modelu: 4 

Mam nadzieję, że
Poprawna odpowiedź: 12
-------------------------------
4 

Mam nadzieję, że
Pytanie: 2 + 15 =
Odpo

PODSUMOWANIE:

- DZIAŁANIA PODZIELONE NA KATEGORIE: EFEKTYWNOŚĆ 25%; USTALENIE JEDNEGO CZYNNIKA W DZIAŁANIU: 35%;

- Z PROMPTEM "DZIAŁANIE ... WYNIK ..." EFEKTYWNOŚĆ 20%; Z USTALONYM CZYNNIKIEM: 35%